# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via a cargo plane to there, and represent them on a map, with a color varying by a cargo plane transfer time. Also represent some supercar factories with the same cargo plane transfer time.

Let's build this!

In [1]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q

In [2]:
from dotenv import load_dotenv
from smolagents import LiteLLMModel

load_dotenv()

MODEL_ID = "ollama_chat/qwen2.5-coder:7b"

model = LiteLLMModel(
    model_id=MODEL_ID,  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192
)

In [3]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [4]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, VisitWebpageTool


# model = InferenceClientModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

We can start with creating a baseline, simple agent to give us a simple report.

In [5]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [6]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [7]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Function to get filming locations and calculate travel times                                                   
  def get_locations_and_travel_times(query):                                                                       
      search_results = web_search(query=query)                                                                     
      print(f"Search Results: {search_results}")                                                                   
                                                                                                                   
      data = []                                                                                                    
      for result in search_results.split("\n"):                                                                    
          if "filming location" in result:                                                                         
              name = result.split(":")[0].strip()                                                                  
              coords = result.split(": ")[1].strip().split(", ")                                                   
              latitude = float(coords[0])                                                                          
              longitude = float(coords[1])                                                                         
                                                                                                                   
              travel_time = calculate_cargo_travel_time(                                                           
                  origin_coords=(40.7128, -74.0060),                                                               
                  destination_coords=(latitude, longitude)                                                         
              )                                                                                                    
                                                                                                                   
              data.append({                                                                                        
                  "Location": name,                                                                                
                  "Latitude": latitude,                                                                            
                  "Longitude": longitude,                                                                          
                  "Travel Time (hours)": travel_time                                                               
              })                                                                                                   
                                                                                                                   
      return pd.DataFrame(data)                                                                                    
                                                                                                                   
  # Get Batman filming locations and their travel times                                                            
  batman_filming_locations = get_locations_and_travel_times("Batman filming locations")                            
                                                                                                                   
  # Get supercar factories and their travel times                                                                  
  supercar_factories = get_locations_and_travel_times("Su

Execution logs:
Search Results: ## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George's Hall, Liverpool, England, UK. (Exterior ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) ; Acton Lane 
Power Station, Acton Lane, Acton, London, England ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqJIAxgNENc73uxFNwqY8vD3m_LraGtC26VG6D
MC_XyH3SLH0VB)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...

Code execution failed at line 'batman_filming_locations = get_locations_and_travel_times("Batman filming 
locations")' due to: ValueError: could not convert string to float: "St George's Hall"

[Step 1: Duration 23.52 seconds| Input tokens: 2,399 | Output tokens: 419]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Function to get filming locations and calculate travel times                                                   
  def get_locations_and_travel_times(query):                                                                       
      search_results = web_search(query=query)                                                                     
      print(f"Search Results: {search_results}")                                                                   
                                                                                                                   
      data = []                                                                                                    
      for result in search_results.split("\n"):                                                                    
          if "filming location" in result:                                                                         
              parts = result.split(":")[1].strip().split(", ")                                                     
              if len(parts) == 2 and all(part.strip().replace('.', '', 1).isdigit() for part in parts):            
                  name = result.split(":")[0].strip()                                                              
                  coords = [float(part.strip()) for part in parts]                                                 
                                                                                                                   
                  travel_time = calculate_cargo_travel_time(                                                       
                      origin_coords=(40.7128, -74.0060),                                                           
                      destination_coords=tuple(coords)                                                             
                  )                                                                                                
                                                                                                                   
                  data.append({                                                                                    
                      "Location": name,                                                                            
                      "Latitude": coords[0],                                                                       
                      "Longitude": coords[1],                                                                      
                      "Travel Time (hours)": travel_time                                                           
                  })                                                                                               
                                                                                                                   
      return pd.DataFrame(data)                                                                                    
                                                                                                                   
  # Get Batman filming locations and their travel times                                                            
  batman_filming_locations = get_locations_and_travel_times("Batman filming locations")                            
                                                                                                                   
  # Get supercar factories and their travel times                                                                  
  supercar_factories = get_locations_and_travel_times("Su

Execution logs:
Search Results: ## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George's Hall, Liverpool, England, UK. (Exterior ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

The Batman filming location: St George's Hall, Liverpool · The Batman filming location: Royal Liver Building, Pier 
Head, Liverpool · The Batman filming location: ...

2. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations ; Knebworth House, Knebworth, Hertfordshire, England, UK. (Wayne Manor; exterior) ; Acton Lane 
Power Station, Acton Lane, Acton, London, England ...

3. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

The Batman was primarily filmed in the United Kingdom. Most of the recognizable buildings of this new version of 
Gotham City are located in Liverpool.

4. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

5. [Where was Batman Begins filmed?](https://giggster.com/guide/movie-location/where-was-batman-begins-filmed)
Source: Giggster

Batman Begins was filmed in various locations across the globe, with most scenes shot in the United Kingdom, 
specifically in London, Buckinghamshire, and Essex.

6. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOoqJIAxgNENc73uxFNwqY8vD3m_LraGtC26VG6D
MC_XyH3SLH0VB)
Date published: Apr 20, 2023
Source: LA Today

Batman Movies and Filming Locations in LA · Cliffs of Marineland · Kirkeby Center · Gateway East Building · Wayne 
Manor · Bronson Caves · Batman ...
Search Results: ## Search Results
0. [Vroom! Touring Italy's Supercar 
Factories](https://www.nytimes.com/2025/03/14/travel/motor-valley-italy-supercars.html)
Date published: Mar 14, 2025
Source: The New York Times

In the country's “Motor Valley,” racecar enthusiasts can admire, and even drive, Maseratis, Lamborghinis, Ferraris 
and more.

1. [Ferrari Lamborghini Pagani Ducati Factory Tours](https://www.italianfactorymotortour.com/)
Source: Italian Factory Motor Tour

ITALIAN FACTORY MOTOR TOUR offer Ferrari Factory tours, Lamborghini Factory tours, Ducati Factory tours, Pagani 
Factory tours, Maserati Museum and Factory ...

2. [List of exclusively sports car 
manufacturers](https://en.wikipedia.org/wiki/List_of_exclusively_sports_car_manufacturers)
Source: Wikipedia

Etox (Turkey) · Ferrari (Italy) · Fittipaldi Motors LLC (Italy) · Foreman (UK) · FYK (Norway) · Gardner Douglas 
(UK) · Gemballa (Germany) · Gillet (Belgium) ...

3. [Italian Supercars](https://www.supercarsmanufacturing.com/)
Source: supercarsmanufacturing.com

Sly Garage Factory srl is one of the most important Italian manufacturer of top Sport Supercars, high end 
automotive industry, mechanical technology for dealers ...

4. [GTM Supercar Kit](https://www.factoryfive.com/gtm-supercar/what-you-get/)
Source: Factory Five Racing

GTM Supercar Kit: $24,990. Some Assembly Required. The GTM Supercar kit is engineered to be built as a chassis kit 
by a car guy at home with regular tools. The ...

5. [Supercar Factory 
Showroom](https://www.letbricks.com/product/supercar-factory-showroom/?srsltid=AfmBOoqmZWSmTVUNTji9NifIosy1Q4NxweN2
q79A0ZDvLN1ibLXouBYd)
Source: Letbricks

Supercar Factory Showroom · Made of high quality, eco-friendly ABS plastic which is odorles

[Step 2: Duration 24.69 seconds| Input tokens: 6,336 | Output tokens: 823]

In [8]:
result

{'Batman Filming Locations': Empty DataFrame
 Columns: []
 Index: [],
 'Supercar Factories': Empty DataFrame
 Columns: []
 Index: []}

We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [9]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## Facts survey
### 1.1. Facts given in the task
- Task requires finding all Batman filming locations.
- Needs to calculate the time to transfer via cargo plane from Gotham (40.7128° N, 74.0060° W) to each filming 
location.
- Needs to return the results as a pandas dataframe.
- Needs to find supercar factories with the same cargo plane transfer time.

### 1.2. Facts to look up
- List of all known Batman filming locations globally (web search, IMDb, Wikipedia).
- Geographic coordinates for each Batman filming location.
- Details about existing supercar factories (web search, automotive industry websites).

### 1.3. Facts to derive
- Calculate the cargo plane transfer time from Gotham to each Batman filming location using the given formula.
- Derive a pandas dataframe with columns: 'Batman_Filming_Location', 'Coordinates', 'Cargo_Transfer_Time'.
- Identify supercar factories that have the same or similar cargo transfer times.

## Plan
1. Perform a web search for "Batman filming locations worldwide" to gather a list of known Batman filming 
locations.
2. Use the `visit_webpage` tool to visit each webpage listed in step 1 and extract geographic coordinates 
(latitude, longitude) for each location.
3. Calculate the cargo plane transfer time from Gotham to each Batman filming location using the 
`calculate_cargo_travel_time` tool with default cruising speed.
4. Create a pandas dataframe with columns: 'Batman_Filming_Location', 'Coordinates', 'Cargo_Transfer_Time' and 
populate it with data derived in steps 2 and 3.
5. Perform another web search for "supercar factories" to gather a list of known supercar manufacturers.
6. Use the `visit_webpage` tool to visit each webpage listed in step 5 and extract details about existing supercar 
factories, focusing on geographic locations.
7. Calculate the cargo plane transfer time from Gotham to each supercar factory using the 
`calculate_cargo_travel_time` tool with default cruising speed.
8. Identify supercar factories that have the same or similar cargo transfer times as Batman filming locations by 
comparing the 'Cargo_Transfer_Time' column of the dataframe created in step 4 with the times calculated in step 7.
9. Provide the final answer as a pandas dataframe containing the relevant information about Batman filming 
locations and supercar factories with matching cargo transfer times.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_query = "Batman filming locations worldwide"                                                    
  batman_locations_results = web_search(query=batman_locations_query)                                              
  print(batman_locations_results)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Category:Film Locations - Batman Wiki](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki | Fandom

Batman Wiki Film Locations Category Page. Locations that appear in the films. Trending pages: The Pit, The Pit, The
Pit, The Narrows, The Narrows, The Narrows.

3. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

4. [Where was The Dark Knight filmed?](https://giggster.com/guide/movie-location/where-was-the-dark-knight-filmed)
Source: Giggster

The Dark Knight locations were spread across the world with filming in Chicago in the United States, London in 
England, and Hong Kong in China.

5. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George's Hall, Liverpool, England, UK. (Exterior ...

6. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

7. [The Filming Locations of The Batman : 
r/Tartaria](https://www.reddit.com/r/Tartaria/comments/tsd5zf/the_filming_locations_of_the_batman/)
Source: Reddit · r/Tartaria

'The Batman 2' is reportedly gearing up to begin filming in London in early 2026. ... r/DC_Cinematic - 'The Batman 
2' is reportedly gearing up to ...

8. [What are the film locations of Batman 
movies?](https://www.facebook.com/groups/3164534807204454/posts/3749390682052194/)
Source: Facebook · Film Locations - Then and Now

Wollaton hall in Nottingham was used as Wayne Manor in the dark knight. 1 yr.

9. [Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
Date published: Mar 4, 2022
Source: Screen Rant

The entirety of Wellington Square in Liverpool, England was used as The Batman's filming location to create what is
known as Gotham Square.

Out: None

...

Execution logs:
## Search Results
0. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Most of the filming of Batman movies is done in the Warner Bros studios and across the US, including New York and 
Pittsburgh. You will notice ...

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Category:Film Locations - Batman Wiki](https://batman.fandom.com/wiki/Category:Film_Locations)
Source: Batman Wiki | Fandom

Batman Wiki Film Locations Category Page. Locations that appear in the films. Trending pages: The Pit, The Pit, The
Pit, The Narrows, The Narrows, The Narrows.

3. [What cities in America and anywhere in the world do you 
...](https://www.reddit.com/r/batman/comments/1d1t88q/what_cities_in_america_and_anywhere_in_the_world/)
Source: Reddit · r/batman

Glasgow and Liverpool were used in the shoot for The Batman. I'd avoid Chicago, as that looks more like Metropolis 
than Gotham.

4. [Where was The Dark Knight filmed?](https://giggster.com/guide/movie-location/where-was-the-dark-knight-filmed)
Source: Giggster

The Dark Knight locations were spread across the world with filming in Chicago in the United States, London in 
England, and Hong Kong in China.

5. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations ; Necropolis Cemetery, Glasgow, Scotland, UK. (Batman and Selina leaving the cemetery) ; St. 
George's Hall, Liverpool, England, UK. (Exterior ...

6. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

7. [The Filming Locations of The Batman : 
r/Tartaria](https://www.reddit.com/r/Tartaria/comments/tsd5zf/the_filming_locations_of_the_batman/)
Source: Reddit · r/Tartaria

'The Batman 2' is reportedly gearing up to begin filming in London in early 2026. ... r/DC_Cinematic - 'The Batman 
2' is reportedly gearing up to ...

8. [What are the film locations of Batman 
movies?](https://www.facebook.com/groups/3164534807204454/posts/3749390682052194/)
Source: Facebook · Film Locations - Then and Now

Wollaton hall in Nottingham was used as Wayne Manor in the dark knight. 1 yr.

9. [Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
Date published: Mar 4, 2022
Source: Screen Rant

The entirety of Wellington Square in Liverpool, England was used as The Batman's filming location to create what is
known as Gotham Square.

Out: None

[Step 13: Duration 75.67 seconds| Input tokens: 83,264 | Output tokens: 2,791]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 14 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 14: Duration 10.18 seconds| Input tokens: 90,454 | Output tokens: 2,864]

KeyboardInterrupt: 

In [ ]:
detailed_report

Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `plotly`, and `geopandas` + `shapely` for spatial plotting.

In [ ]:
# model = InferenceClientModel(
#     "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
# )

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(provider="serper"),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [ ]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=model,
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

Let us inspect what this team looks like:

In [ ]:
manager_agent.visualize()

In [ ]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

I don't know how that went in your run, but in mine, the manager agent skilfully divided tasks given to the web agent in `1. Search for Batman filming locations`, then `2. Find supercar factories`, before aggregating the lists and plotting the map.

Let's see what the map looks like by inspecting it directly from the agent state:

In [ ]:
manager_agent.python_executor.state["fig"]

![output map](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/unit2/smolagents/output_map.png)